In [1]:
import json
import random

import emoji
import numpy as np
import pandas as pd
import torch
from datasets import load_metric
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

PyTorch version 1.12.0+cu102 available.
2023-06-04 12:29:03.262829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 12:29:03.627829: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-9.0/lib64:/usr/local/cuda-9.0/lib64:
2023-06-04 12:29:03.627878: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-04 12:29:05.440947: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libra

In [2]:
train_df = pd.read_csv("train/train.En.csv")[["tweet", "sarcastic", "rephrase"]]
train_df["tweet"] = train_df["tweet"].fillna("")
train_df["rephrase"] = train_df["rephrase"].fillna("")
train_df

,tweet,sarcastic,rephrase
0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring..."
1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...
2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall..."
3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to..."
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...
...,...,...,...
3463,The population spike in Chicago in 9 months is...,0,
3464,You'd think in the second to last English clas...,0,
3465,I’m finally surfacing after a holiday to Scotl...,0,
3466,Couldn't be prouder today. Well done to every ...,0,


In [3]:
test_df = pd.read_csv("test/task_A_En_test.csv")
test_df["text"] = test_df["text"].fillna("")
test_df

,text,sarcastic
0,"Size on the the Toulouse team, That pack is mo...",0
1,Pinball!,0
2,So the Scottish Government want people to get ...,1
3,villainous pro tip : change the device name on...,0
4,I would date any of these men 🥺,0
...,...,...
1395,I’ve just seen this and felt it deserved a Ret...,0
1396,Omg how an earth is that a pen !!! 🤡,0
1397,Bringing Kanye and drake to a tl near you,0
1398,"I love it when women are referred to as ""girl ...",1


In [4]:
acc_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
thd = 0.25


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    y_pred = torch.softmax(torch.FloatTensor(predictions), dim=1)[:, 1] > thd
    scores = f1_metric.compute(predictions=y_pred, references=labels)
    scores.update(acc_metric.compute(predictions=y_pred, references=labels))
    return scores

Checking /home/ray080186/.cache/huggingface/datasets/4c530db6524e184f3ae54c23f61ccff4ed3044ff6415e1e28d05618f6eb71752.f7b1a1b5c5768ef71463744e19ab26eb2bfa9b9d2ca9ece6a94ee05f37520247.py for additional imports.
Found main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/datasets/metrics/accuracy/accuracy.py at /home/ray080186/.cache/huggingface/modules/datasets_modules/metrics/accuracy
Found specific version folder for metric https://s3.amazonaws.com/datasets.huggingface.co/datasets/metrics/accuracy/accuracy.py at /home/ray080186/.cache/huggingface/modules/datasets_modules/metrics/accuracy/618004201c82ad7cefba547c6116ab7245ee7ccf428d0b23483d20bcc118fb51
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/datasets/metrics/accuracy/accuracy.py to /home/ray080186/.cache/huggingface/modules/datasets_modules/metrics/accuracy/618004201c82ad7cefba547c6116ab7245ee7ccf428d0b23483d20bcc118fb51/accuracy.py
Couldn't find dataset infos file at https://s3.amazona

In [5]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return emoji.demojize(" ".join(new_text), language="en")

### Fine-tuned Transformers
***

#### Customized dataset

In [6]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#### Training pipeline

In [7]:
def train_and_evaluate(model_id="bert-base-cased", seed=0):
    print(f"{model_id=}, {seed=}")
    
    # set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # prepare training data
    all_idxs = np.arange(len(train_df))
    train_idxs, valid_idxs = get_train_test_indices(len(all_idxs), test_size=0.2)
    train_idxs, valid_idxs = all_idxs[train_idxs], all_idxs[valid_idxs]
    X_train = train_df["tweet"].apply(preprocess).values[train_idxs]
    X_valid = train_df["tweet"].apply(preprocess).values[valid_idxs]
    X_test = test_df["text"].apply(preprocess).values
    y_train = train_df["sarcastic"].values[train_idxs]
    y_valid = train_df["sarcastic"].values[valid_idxs]
    y_test = test_df["sarcastic"].values
    
    # tokenize
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
    valid_encodings = tokenizer(list(X_valid), truncation=True, padding=True, max_length=512)
    test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)
    
    # prepare torch dataset
    train_dataset = TweetDataset(train_encodings, y_train)
    valid_dataset = TweetDataset(valid_encodings, y_valid)
    test_dataset = TweetDataset(test_encodings, y_test)
    
    # fine-tune given transformer
    batch_size = 16
    gradient_accumulation_steps = 1
    actual_batch_size = batch_size * gradient_accumulation_steps
    n_epoch = 3
    total_steps = round(len(y_train) / actual_batch_size / 4 * n_epoch)
    warmup_steps = total_steps * 0.1

    training_args = TrainingArguments(
        output_dir="/tmp2/kch/cs263/final_project/models",
        learning_rate=1e-4,
        num_train_epochs=n_epoch,                # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        gradient_accumulation_steps=gradient_accumulation_steps,
        per_device_eval_batch_size=64,           # batch size for evaluation
        warmup_steps=warmup_steps,               # number of warmup steps for learning rate scheduler
        weight_decay=0.01,                       # strength of weight decay
        logging_dir="./logs",                    # directory for storing logs
        logging_steps=20,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model="f1",
        load_best_model_at_end=True,
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=2,
        ignore_mismatched_sizes=True,
    )

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
        compute_metrics=compute_metrics,
    )

    trainer.train()
    
    # evaluate
    y_pred = trainer.predict(test_dataset)[0]
    y_pred = torch.softmax(torch.FloatTensor(y_pred), dim=1)[:, 1] > thd
    renamed_model_id = model_id.replace("/", "-")
    with open(f"predictions/{renamed_model_id}_{seed}.npy", "wb") as f:
        np.save(f, y_pred.detach().cpu().numpy())
    return f1_metric.compute(predictions=y_pred, references=y_test)["f1"]


def get_train_test_indices(n, test_size):
    n_train = round(n * (1 - test_size))
    all_idxs = list(range(n))
    random.shuffle(all_idxs)
    return all_idxs[:n_train], all_idxs[n_train:]

In [8]:
model_ids = [
    "roberta-base",
    "roberta-large",
    "cardiffnlp/twitter-roberta-base-sentiment",
    "cardiffnlp/twitter-roberta-base-irony",
    "jkhan447/sarcasm-detection-RoBerta-base-CR",
    "helinivan/english-sarcasm-detector",
    "brahimje/roberta-sarcasm-detection",
]
model_id2f1 = {}
k = 10


for i, model_id in enumerate(model_ids):
    f1s = []
    for j in range(k):
        seed = j + 5497
        f1 = train_and_evaluate(model_id=model_id, seed=seed)
        print("F1:", round(f1, 3))
        f1s.append(f1)
    model_id2f1[model_id] = round(np.mean(f1s), 3)

model_id='brahimje/roberta-sarcasm-detection', seed=5497


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at brahimje/roberta-sarcasm-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
***** Running training *****
  Num examples = 2774
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 132


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.334300,0.224160,0.808864,0.900576
2,0.188400,0.216571,0.831250,0.922190
3,0.048700,0.409479,0.788732,0.913545


***** Running Evaluation *****
  Num examples = 694
  Batch size = 256
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Saving model checkpoint to /tmp2/kch/cs263/final_project/models/checkpoint-44
Configuration saved in /tmp2/kch/cs263/final_project/models/checkpoint-44/config.json
Model weights saved in /tmp2/kch/cs263/final_project/models/checkpoint-44/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 256
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/hug

Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.


...

loading file https://huggingface.co/brahimje/roberta-sarcasm-detection/resolve/main/vocab.json from cache at /home/ray080186/.cache/huggingface/transformers/93bebd4410749f5d4f51af40d0ce91f8551fa116d2327d62c181902d6bfc542b.bfdcc444ff249bca1a95ca170ec350b442f81804d7df3a95a2252217574121d7
loading file https://huggingface.co/brahimje/roberta-sarcasm-detection/resolve/main/merges.txt from cache at /home/ray080186/.cache/huggingface/transformers/b8385573b031c988fd5e01524c0b67ccce412a56470f2160559df11116fd1a7a.f5b91da9e34259b8f4d88dbc97c740667a0e8430b96314460cdb04e86d4fc435
loading file https://huggingface.co/brahimje/roberta-sarcasm-detection/resolve/main/tokenizer.json from cache at /home/ray080186/.cache/huggingface/transformers/31e20f68b315cad181b7fe550df7a3480761f4d1fe14911b39f009937a87b2e1.cfc2bbe974cb2cd49fa76285296a2ded2cbfe78d9ca0705ea6527d434f182f0b
loading file https://huggingface.co/brahimje/roberta-sarcasm-detection/resolve/main/added_tokens.json from cache at None
loading file h

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.351100,0.229772,0.835165,0.913545
2,0.172300,0.338225,0.793750,0.904899
3,0.032200,0.545073,0.765517,0.902017


***** Running Evaluation *****
  Num examples = 694
  Batch size = 256
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Saving model checkpoint to /tmp2/kch/cs263/final_project/models/checkpoint-44
Configuration saved in /tmp2/kch/cs263/final_project/models/checkpoint-44/config.json
Model weights saved in /tmp2/kch/cs263/final_project/models/checkpoint-44/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 256
Done writing 694 examples in 5552 bytes /home/ray080186/.cache/hug

Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Done writing 1400 examples in 11200 bytes /home/ray080186/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow.
Set __getitem__(key) output type to python objects for no columns  (when key is int or slice) and don't output other (un-formatted) columns.


F1: 0.57


In [9]:
print(json.dumps(model_id2f1, indent=2))

{
  "brahimje/roberta-sarcasm-detection": 0.747
}
